In [2]:
import pandas as pd

In [3]:
data = pd.read_csv("../data/2021-10-19_14-11-08_val_candidate_data.csv")

In [4]:
data.head()

,Unnamed: 0,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,feature_11,feature_12,target
0,298,Female,No,0,Graduate,No,5000.0,2541,151.0,480.0,1.0,Rural,0,No,N
1,115,Male,Yes,1,Graduate,No,14583.0,0,185.0,180.0,1.0,Rural,1,No,Y
2,312,Female,No,0,Graduate,No,2507.0,0,56.0,360.0,1.0,Rural,0,No,Y
3,605,Male,Yes,0,Not Graduate,No,2400.0,3800,NaN,180.0,1.0,Urban,0,No,N
4,465,Male,No,0,Graduate,No,6000.0,0,140.0,360.0,1.0,Rural,0,No,Y
